In [ ]:
#---------------------------------- IMPORTING!! ---------------------------------- 

%matplotlib notebook
import rebound
import numpy as np
import sys
from IPython.display import display, clear_output
from math import sqrt, log10, sin, cos, atan
import random as rnd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

In [ ]:
#---------------------------------- DEFINING!! ---------------------------------- 

# All distances in A.U.
mh = 4.0e6
mb = 3.76e9/mh
md = 6.0e10/mh
mha = 1.0e12/mh
ab = 2.0e7
rh = 4.125e9
ad = 5.7e8
bd = 6.2e7
scale = 1.6e9

# Galaxy potential taken from http://adsabs.harvard.edu/abs/2014ApJ...793..122K,
# Note: There is a typo in that paper where "a_d" is said to be 2750 kpc, it should be 2.75 kpc.

def migrationAccel(reb_sim):
    r = sqrt(ps[1].x**2+ps[1].y**2+ps[1].z**2)
    rho2 = ps[1].x**2+ps[1].y**2
    zbd = sqrt(ps[1].z**2+bd**2)
    logterm = np.log(1.0+(r/rh))
    rterm = (r+rh)*(r**2)
    
    func = lambda r,c,k: ((1.0/np.pi)*atan((r-c)/k) + 0.5)
    f3 = lambda r: func(r,1.0e6,1.0e2)
    
    ps[1].ax += -mb*ps[1].x/(r*(ab+r)**2) -\
                md*ps[1].x/(rho2+(ad+zbd)**2)**1.5 #+\
                #f3(r)*mha*ps[1].x*((logterm/r**3)-(1.0/rterm))
    ps[1].ay += -mb*ps[1].y/(r*(ab+r)**2) -\
                md*ps[1].y/(rho2+(ad+zbd)**2)**1.5 #+\
                #f3(r)*mha*ps[1].y*((logterm/r**3)-(1.0/rterm))
    ps[1].az += -mb*ps[1].y/(r*(ab+r)**2) -\
                md*ps[1].y/(rho2+(ad+zbd)**2)**1.5 #+\
                #f3(r)*mha*ps[1].z*((logterm/r**3)-(1.0/rterm))
            
# xi = #constant set by local stellar density

# # from Salpeter's equation: 
# # Xi(m) =  xi/(m**2.35)
# # CDF(x) = xi/(x**1.35) when we integrate Xi(m) from -infty to x, and 
# #produces a probability between [0,1].
# # star_func is found by solving for m when given a probability 0 <= x <= 1:
# # this outputs a star mass measured in solar masses
# star_func = lambda x: (x/xi)**1.35 


In [ ]:
#---------------------------------- INTEGRATING!! ---------------------------------- 
Nstars = 2
#nfrag = rnd.randint(100, 300)
nfrag = 10
Nout = 100

t_start = 0
t_end = 10000

posx = [[[] for y in range(nfrag)] for x in range(Nstars)]
posy = [[[] for y in range(nfrag)] for x in range(Nstars)]
posz = [[[] for y in range(nfrag)] for x in range(Nstars)]

for star in tqdm(range(Nstars)):
    # Randomly drawn mass of star disrupted, fraction of star mass lost in disruption, 
    # number of fragments, and initial 3D angle of outgoing stellar debris

#     m_star = star_func(rnd.random())
#     print m_star
#     percentage_lost = rnd.random()
#     m_deb = m_star*percentage_lost
#     m_frag = m_deb/nfrag

    # Velocities are set to be just a little bit hyperbolic.
    vels = [sqrt(2.0 + 1.5e-4*(0.5 + float(x)/float(nfrag))) for x in range(nfrag)]
    for i, vel in enumerate(vels):
        # Set up simulation
        sim = rebound.Simulation()
        sim.integrator = "ias15"
        sim.add(m=1.0)
        
        #set position of fragment; random sphere point picking
        u = rnd.uniform(-1.0,1.0)
        th = rnd.uniform(0., 2.*np.pi)
        phi = rnd.uniform(0., 2.*np.pi)
        
        xx = sqrt(1-(u)**2)*cos(th)
        yy = sqrt(1-(u)**2)*sin(th)
        zz = u
        r = xx**2 + yy**2 + zz**2
        
        #velocity vector functions
        s = lambda r,z: sqrt(2-(2*z/r))*(r**2)
        f = lambda a,b,z,p,r: ((a**2)*b*cos(p))/s(r,z) #a^2*b
        g = lambda a,z,p,r: (a*(z**2)*cos(p))/s(r,z) #a*z^2
        h = lambda a,b,z,p,r: (a*b*sin(p))/s(r,z) #a*b
        j = lambda a,z,p,r: (a*z*sin(p))/s(r,z) #a*z
        k = lambda a,z,p,r: ((a**3)*cos(p))/s(r,z) #a^3
        m = lambda a,z,p,r: ((a**2)*sin(p))/s(r,z) #a^2
        
        velx = lambda x,y,z,p,r: -f(x,y,z,p,r)-k(y,z,p,r)-g(y,z,p,r)-j(x,z,p,r)
        vely = lambda x,y,z,p,r: k(x,z,p,r)+f(y,x,z,p,r)+g(x,z,p,r)+j(y,z,p,r)
        velz = lambda x,y,z,p,r: m(x,z,p,r)+m(y,z,p,r)
        
        #adding particle
        sim.add(m=0.0,x=xx,y=yy,z=zz,vx=vel*velx(xx,yy,zz,phi,r),\
                vy=vel*vely(xx,yy,zz,phi,r),vz=vel*velz(xx,yy,zz,phi,r))
        sim.N_active = 1
        sim.additional_forces = migrationAccel

        ps = sim.particles

        times = np.linspace(0.0, 20.0*((ab)**(1.5)/sqrt(mb))*2.*np.pi, Nout)
        for ti, time in enumerate(times):
            #print(time)
            sim.integrate(time, exact_finish_time=1)
            posx[star][i].append(ps[1].x/scale)
            posy[star][i].append(ps[1].y/scale)
            posz[star][i].append(ps[1].z/scale)
    
    t_start += 10000
    t_end += 10000

In [ ]:
#---------------------------------- PLOTTING!! ---------------------------------- 

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
colors = sns.color_palette("hls", Nstars).as_hex()
for star in range(Nstars):
    for i in range(nfrag):
        ax.plot(posx[star][i],posy[star][i],posz[star][i], c = colors[star]);
        ax.scatter([posx[star][i][-1]],[posy[star][i][-1]],[posz[star][i][-1]],c = colors[star]);

In [ ]:
#---------------------------------- VISUALIZING!! ---------------------------------- 
data = []
for star in range(Nstars):
    for i in range(nfrag):
        end = len(posx[star][i])
        point = [log10(sqrt(posx[star][i][end - 1]**2 + posy[star][i][end - 1]**2)), posz[star][i][end - 1]]
        data.append(point)
        
fragdata = np.array([point for point in data])
df = pd.DataFrame(fragdata, columns=["s", "z"])

#Scatter plots
sns.jointplot(x="s", y="z", data=df);

#Density plots
# sns.jointplot(x="x", y="y", data=df, kind="kde");

g = sns.jointplot(x="s", y="z", data=df, kind="kde", color="m")
g.plot_joint(plt.scatter, c="b", s=10, marker="o")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$\log_{10}(S)$, $S = X^2 + Y^2$", "$Z$");